Use threading.Thread(target=...) to define the task and .start() to kick it off. To make the main program wait for the thread to finish, use .join().

In [1]:

import threading
import time

def boil_water(sec):
    print("starting boiling")
    time.sleep(sec)
    print("boiling done")

t = threading.Thread(target=boil_water, args=(2,))
t.start()
print("\ngetting the cups ready with coffee")
t.join()
print("coffee ready")


starting boiling

getting the cups ready with coffee
boiling done
coffee ready


In [2]:
from queue import Queue
from threading import Thread
import time
import random

def data_fetcher(outp):
    for i in range(5):
        score = random.randint(1,100)
        print(f"[fetcher] found score: {score}")
        outp.put(score)
        time.sleep(1)
    outp.put("end")

def display(inp_pip):
    while True:
        data = inp_pip.get()
        if data == 'end':
            print("ending session")
            break
        print(f"[Printing:] {data}")


sharedq= Queue()
t1 = Thread(target=data_fetcher, args=(sharedq,))
t2 = Thread(target=display, args = (sharedq,))

t1.start()
t2.start()
t1.join()
t2.join()


[fetcher] found score: 83
[Printing:] 83
[fetcher] found score: 84
[Printing:] 84
[fetcher] found score: 40
[Printing:] 40
[fetcher] found score: 85
[Printing:] 85
[fetcher] found score: 78
[Printing:] 78
ending session


In [3]:
class Account:
    def __init__(self,name, balance):
        self.name, self.balance = name, balance
        self.lock = threading.Lock()
    def __str__(self):
        return self.name

def transfer(from_ac, to_ac, amnt):
    locks = sorted([from_ac.lock, to_ac.lock], key = id)

    print(f"Attempting transfer: {from_ac} -> {to_ac} (${amnt})")

    with locks[0]:
        with locks[1]:
            if from_ac.balance >= amnt:
                from_ac.balance -= amnt
                to_ac.balance+=amnt

                time.sleep(1)
                print(f"Successfuly transfered (${amnt})")
                print("new balances: ", f"{from_ac.balance} , {to_ac.balance}")
            else:
                print("Not enough balance")

acc_a = Account('Haseeb', 10000)
acc_b = Account("Tayyab", 10000)
            
t1 = threading.Thread(target=transfer, args=(acc_a, acc_b, 325))
t2 = threading.Thread(target=transfer, args=(acc_b, acc_a, 333))

t1.start()
t2.start()

t1.join()
t2.join()






Attempting transfer: Haseeb -> Tayyab ($325)Attempting transfer: Tayyab -> Haseeb ($333)

Successfuly transfered ($333)
new balances:  9667 , 10333
Successfuly transfered ($325)
new balances:  10008 , 9992


In [7]:
thread_data = threading.local()

def show_user():
    name = threading.current_thread().name
    thread_data.user = name
    time.sleep(1)
    print(f"Data in {name}: {thread_data.user}")

threads = [threading.Thread(target = show_user) for _ in range(4)]
for t in threads: t.start()

Data in Thread-19 (show_user): Thread-19 (show_user)
Data in Thread-16 (show_user): Thread-16 (show_user)
Data in Thread-17 (show_user): Thread-17 (show_user)
Data in Thread-18 (show_user): Thread-18 (show_user)


In [10]:
from collections import defaultdict

class Exchange:
    def __init__(self):
        self._subscribers = set()
    
    def attach(self, task):
        self._subscribers.add(task)
    def detach(self, task):
        self._subscribers.remove(task)

    def send(self, msg):
        for subscriber in self._subscribers:
            subscriber(msg)

def police_rad(msg):
    print(f"police recieving: {msg}")
def citizen_radio(msg):
    print(f"Citizen recieving: {msg}")

exc = Exchange()
exc.attach(police_rad)
exc.attach(citizen_radio)

exc.send("Traffic is jammed. everybody go homes")

police recieving: Traffic is jammed. everybody go homes
Citizen recieving: Traffic is jammed. everybody go homes


In [12]:
def task_a():
    for i in range(1,4):
        print(f"task a is running at: {i}")
        yield

def task_b():
    for i in range(21,25):
        print(f"task b is running at: {i}")
        yield


tasks = [task_a(), task_b()]
while tasks:
    task = tasks.pop(0)
    try:
        next(task)
        tasks.append(task)
    except StopIteration:
        pass


task a is running at: 1
task b is running at: 21
task a is running at: 2
task b is running at: 22
task a is running at: 3
task b is running at: 23
task b is running at: 24
